# RAG Q&A LLM Chatbot for [Confluence Space](https://templates.atlassian.net/wiki/spaces/SWPRJ) and Github Repositories ([Full-Stack App](https://crowd-wire.github.io/ProjectDocumentation/), [SQL Tables](https://github.com/JannikArndt/sql-auto-doc/tree/master/Examples))

# Dependencies

In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers atlassian-python-api markdownify lark chromadb opensearch-py llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.6/168.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.1/905.1 kB 51.

# Model Initialization

In [3]:
hf_auth = 'your-hf-key-here'

In [5]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items with hf auth token
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

# enable evaluation mode to allow model inference
model.eval()

print(f"Model loaded on {device}")

Quantization Pros:
- Using smaller precision representations require less memory, which simplifies the deployment of models on devices with limited resources.
- On hardware with efficient integer operations, such as CPUs, GPUs, and TPUs, quantized models execute quickly.
- On mobile devices and in data centers, lower-precision computations are more energy-efficient.

Quantization Cons:
- Due to precision loss, quantization might diminish model accuracy and negatively affect task performance.
- The process of quantization is intricate and lengthy, requiring additional training steps.

# Tokenizer

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

# Stop On Tokens

In [ ]:
stop_list = ['\nHuman:', '\n```\n']

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

In [ ]:
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

# Text Generation (without RAG)

In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.2,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

# checking again that everything is working fine
# llm(prompt="What is CrowdWire")

# Loading the Documents

In [ ]:
import os
import re
import uuid
from langchain.document_loaders import ConfluenceLoader, DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from llama_index import Document

# Confluence Loader (will transform later the web pages to markdown format)
confluence_loader = ConfluenceLoader(
    url="https://templates.atlassian.net/wiki"
)

headers_to_split_on = [
    ("#", "title"),
    ("##", "header"),
    ("###", "header"),
    ("####", "header"),
    ("#####", "header"),
    ("######", "header"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000, chunk_overlap=200)

def clean_text(text):
    # remove words with more than 100 characters
    text = re.sub(r'\b\w{101,}\b', ' ', text)
    # remove extra white spaces
    text = re.sub(r' +', ' ', text)
    return text

# final documents array
documents = []

## Confluence loader

In [ ]:
# Confluence Loader (will transform later the web pages to markdown format)
confluence_loader = ConfluenceLoader(
    url="https://templates.atlassian.net/wiki"
)

if callable(confluence_loader.load):
  for doc in confluence_loader.load(
      space_key="SWPRJ",
      include_attachments=False,
      keep_markdown_format=True
    ):
    link = doc.metadata["source"]
    source = link.split("/")[-1]
    title = doc.metadata["title"]

    doc.page_content = clean_text(doc.page_content)
    docs = markdown_splitter.split_text(doc.page_content)
    docs = text_splitter.split_documents(docs)

    for doc in docs:
      doc.metadata["link"] = link
      doc.metadata["source"] = source
      doc.metadata["title"] = title
      if "header" not in doc.metadata:
        doc.metadata["header"] = title
      documents.append(Document(
          text = doc.page_content,
          metadata=doc.metadata,
          excluded_llm_metadata_keys = ["link"],
          excluded_embed_metadata_keys = ["link"]
        ))

In [ ]:
len(documents)

## Repository Loader

In [ ]:
# Repository Loader (Markdowns only for now)
glob_pattern = "*.md"
repo_paths = ["/content/drive/MyDrive/ProjectDocumentation-master",
              "/content/drive/MyDrive/SQL_Examples"]
repo_links = ["https://github.com/Crowd-Wire/ProjectDocumentation/tree/master/",
              "https://github.com/JannikArndt/sql-auto-doc/tree/master/Examples/"]

repo_i = 0
for repo_path in repo_paths:
  try:
    repo_loader = DirectoryLoader(
      repo_path,
      glob=glob_pattern,
      show_progress=True,
      recursive=True,
      loader_cls=TextLoader,
      loader_kwargs={"encoding": "utf8"})
  except Exception as e:
    print(f"Error loading files with pattern '{glob_pattern}': {e}")
    break


  if callable(repo_loader.load):
    for doc in repo_loader.load():
      relative_path = os.path.relpath(doc.metadata['source'], repo_path)
      link = repo_links[repo_i] + "/" + relative_path

      doc.page_content = clean_text(doc.page_content)
      docs = markdown_splitter.split_text(doc.page_content)
      docs = text_splitter.split_documents(docs)

      for doc in docs:
        if "title" in doc.metadata:
          title = doc.metadata["title"]
          break

      for doc in docs:
        doc.metadata['link'] = link
        doc.metadata['source'] = relative_path
        doc.metadata['title'] = title
        if "header" not in doc.metadata:
          doc.metadata["header"] = title
        documents.append(Document(
          text = doc.page_content,
          metadata=doc.metadata,
          excluded_llm_metadata_keys = ["link"],
          excluded_embed_metadata_keys = ["link"]
        ))
    repo_i += 1

del docs

## Gathered Documents

In [ ]:
len(documents)

In [ ]:
def pretty_print_docs(docs):
  print(f"\n{'-' * 100}\n".join([f"Document {i+1} \n\
Title: {d.metadata['title']}\n\
Header: {d.metadata['header']}\n\
Source: {d.metadata['source']}\n\
Link: {d.metadata['link']}\n\
Content:\n{d.text}" for i, d in enumerate(docs)]))

def pretty_print_answer(answer):
  print(f"Answer: {answer}\n\n")

def pretty_print_result(result):
  print(f"Question: {result['query']}\n\n")
  print(f"Answer: {result['result']}\n\n")
  print("-" * 100)
  print("Documents:\n")
  pretty_print_docs(result["source_documents"])

def pretty_print_response(question):
  response = query_engine.query(question)
  print(f"Question: {question}\n\n")
  print(f"Answer: {response.response}\n\n")
  print("-" * 100)
  print("Documents:\n")
  pretty_print_docs(response.source_nodes)

In [ ]:
pretty_print_docs(documents)

# Creating Embeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index import ServiceContext, LLMPredictor, LangchainEmbedding, set_global_service_context

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cuda"}
  )
)
llm_predictor = LLMPredictor(llm=llm)

service_context = ServiceContext.from_defaults(
  llm_predictor=llm_predictor,
  embed_model=embed_model,
)

# Preparing the Retrieval QA LLM with RAG

In [ ]:
from llama_index import Prompt, VectorStoreIndex
from llama_index.response_synthesizers import get_response_synthesizer
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.vector_stores.types import MetadataInfo, VectorStoreInfo

similarity_top_k = 6
mmr_threshold = 0.7

# define a prompt template
text_qa_template = Prompt("""You are a QA Bot. Based on the documents answer the question at the end. Don't make up an answer if you don't know the answer.

{context_str}

Question: {query_str}

Answer: """)


# build index
index = VectorStoreIndex.from_documents(
  documents,
  service_context=service_context,
  similarity_top_k=similarity_top_k,
  vector_store_query_mode="mmr",
  vector_store_kwargs={"mmr_threshold": mmr_threshold}
)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=similarity_top_k,
    vector_store_query_mode="mmr",
    vector_store_kwargs={"mmr_threshold": mmr_threshold},
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
  text_qa_template=text_qa_template,
  service_context=service_context,
  response_mode="compact"
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# Question Answering

## [Confluence Space](https://templates.atlassian.net/wiki/spaces/SWPRJ)

In [ ]:
pretty_print_response("What are the confluence user story dashboard requirements?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What are the confluence user story dashboard requirements?


Answer:  The Confluence user story dashboard requirements are as follows:

1. As a user, I want to see the most recent activity related to my work so that I can efficiently triage my work for the day.
2. As a user, I want to see comments in the activity feed so that I can see what someone has said without going all the way to a page.
3. As a user, I want to be able to save a page for later so that I can quickly go back to it on my desktop.
4. As a user, I want to share a page so that I can let my colleagues know to check it out.
5. As a user, I want a timestamp on the activity feed items so that I can see how recent they are.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: User story: Dashboard
Header: Primary user story
Source: User+story%3A+Dashboard
Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/33674/User+story%3

In [ ]:
pretty_print_response("What is and what is the summary of the confluence 2016-05-23 Retrospective")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What is and what is the summary of the confluence 2016-05-23 Retrospective


Answer:  The summary of the confluence 2016-05-23 Retrospective is as follows:

* 75% of issues were resolved and deployed.
* New QA process helped eliminate last-minute blocking bugs.
* Designs were finalized and ready in advance of starting the sprint.
* Feedback on pull requests was added within a day.
* No critical bugs were introduced.
* Build issues prevented us from resolving them and will be moved to a subsequent sprint.
* Back-end development took more time than expected and needs to be considered in future estimations.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: 2016-05-23 Retrospective
Header: Retrospective
Source: 2016-05-23+Retrospective
Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/33560/2016-05-23+Retrospective
Content:
#91c89c#f3f9f4### What did we do well?  
* 75% of issues were 

In [ ]:
pretty_print_response("What projects does the confluence space has?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What projects does the confluence space has?


Answer: 
The confluence space has the following projects:

* Demo Project
* Search
* Retrospectives
* Decision Log

Note: These are the projects that are available in the sample data provided in the source documents.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: What is this site ?
Header: What is this site ?
Source: 1972475
Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/1972475
Content:
We've created this site to enable you to see how a Confluence site looks like.  
You can visit this site just as if it was yours, except that you are in read only, so you're not able to modify the content.  
This site is still beta and our teams are working on it in order to display as many features as we can.
----------------------------------------------------------------------------------------------------
Document 2 
Title: 2016-05-12 Sprint

In [ ]:
pretty_print_response("How are search results ordered?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: How are search results ordered?


Answer:  According to the document, search results are ordered based on the following options:

* By date (surfaces the most recent content)
* In alphabetical order (easy to organize and find what you're looking for if you know the title, but not useful if the title is unknown)
* By popularity (surfaces the most important current content, but may not be useful if users are searching for something obscure)

It is also mentioned that the decision on how to order search results was made based on user research and analytics.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: How should search results be ordered?
Header: Background
Source: 33510
Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/33510
Content:
The 'Search' function is used to find relevant pages, blogs, files and comments on Confluence. These display the page, blog or file name, the space

In [ ]:
pretty_print_response("What is planned for notifications?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What is planned for notifications?


Answer:  Based on the provided documents, it appears that the goal for notifications is to implement a working prototype. The team is also considering what the notification feed blank state should look like, and there are several options being considered, including an empty blue background, Confluence pro-tips, and a funny illustration of a puppy looking for content. Additionally, the team is deciding whether to include blog posts in the notification feed.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: 2016-06-19 Sprint Planning "Notifications"
Header: Goals
Source: 33655
Link: https://templates.atlassian.net/wiki/spaces/SWPRJ/pages/33655
Content:
grey1dashed### In this sprint we aim to implement a working prototype for notifications.  
---
----------------------------------------------------------------------------------------------------
Document 2 
Ti

In [ ]:
pretty_print_response("What are some questions that the the confluence space answers?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What are some questions that the the confluence space answers?


Answer:  Based on the provided sources, the Confluence space answers various questions related to decision-making, retrospectives, sprint planning, and notification feeds. Some of the questions answered by the Confluence space include:

* What is this site?
* How do I create a decision?
* How do I create a retrospective?
* Should blog posts be included in the notification feed?
* How do I plan my meetings and share notes and actions with my team?

The Confluence space also provides answers to questions related to the structure and organization of the platform, such as how to create a new space or how to manage permissions. However, it does not provide answers to questions outside the scope of the Confluence platform, such as general knowledge or industry-specific topics.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: What is t

## [Full-Stack App GH Repo](https://crowd-wire.github.io/ProjectDocumentation/)

In [ ]:
pretty_print_response("What is CrowdWire?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What is CrowdWire?


Answer:  Based on the provided documents, CrowdWire appears to be a web-based platform designed for massive online meetings. It utilizes various technologies such as FastAPI, ReactJS, and Mediasoup to provide features like world-editor, full integration of each micro-service, proximity video and voice chatting with high availability, CI/CD, and file exchange. Additionally, the platform allows users to run it locally by executing certain commands inside the `frontend/` directory.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: CrowdWire - Massive Online Meetings
Header: 3rd Milestone
Source: README.md
Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//README.md
Content:
- World-Editor
- Full Integration of each micro-service
- Proximity video and voice chatting with high availability
- CI/CD
- File Exchange feature
--------------------------------------

In [ ]:
pretty_print_response("What is the structure of CrowdWire kubernets charts?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What is the structure of CrowdWire kubernets charts?


Answer:  The structure of CrowdWire kubernetes charts is organized into an umbrella chart containing all microservices as subcharts. The subcharts include:

* api
* frontend
* rabbitmq
* postgresql
* redis

Note: The `rabbitmq`, `postgresql`, and `redis` charts belong to Bitnami.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: Welcome to CrowdWire
Header: Run our Helm Charts
Source: docs/deployment.md
Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs/deployment.md
Content:
To manage the packaging of the Kubernetes resources, we are using [Helm]( https://helm.sh/). If you want to make use of our charts locally, do the following steps. Before installing this chart please check the previous deployment notes.  
```
$cd charts

$helm dep build

$helm install crowdwire .
```  
To observe the initialization of all Pods,

In [ ]:
pretty_print_response("What are the technologies stack used in CrowdWire?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What are the technologies stack used in CrowdWire?


Answer:  Based on the document provided, the following technologies are used in CrowdWire:

* Backend REST API with FastAPI
* Frontend website with ReactJS
* Configuration files for deployment with Kubernetes (k8s)
* Media server for communications with Mediasoup
* World-Editor
* Proximity video and voice chatting with high availability
* CI/CD
* File Exchange feature

Please note that this list may not be exhaustive as there might be other technologies used in CrowdWire that are not mentioned in the provided documents.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: CrowdWire - Massive Online Meetings
Header: 1st Milestone
Source: README.md
Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//README.md
Content:
- Set up docs platforms tools
- Develop a prototype
- Research technologies and define the system architecture
-

In [ ]:
pretty_print_response("What is the Crowdwire media server framework?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What is the Crowdwire media server framework?


Answer:  The Crowdwire media server framework is a Selective Forwarding Unit (SFU) topology that routes media between participants while balancing its limitations with the media inputs it receives. It uses Mediasoup as its server side library and client side library for interaction within the browser.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: Welcome to CrowdWire
Header: Overview
Source: docs/media_server.md
Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs/media_server.md
Content:
[Mediasoup](https://mediasoup.org/) has an SFU (Selective Forwarding Unit) topology, where the server routes media around between participants while balancing its limitations with the media inputs it receives. With this approach, participants send their media to the server and receive others' media in separate streams, one each.  
For m

In [ ]:
pretty_print_response("What is the Mediasoup website link?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What is the Mediasoup website link?


Answer:  The Mediasoup website link is <https://mediasoup.org/>.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: Welcome to CrowdWire
Header: Overview
Source: docs/media_server.md
Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs/media_server.md
Content:
[Mediasoup](https://mediasoup.org/) has an SFU (Selective Forwarding Unit) topology, where the server routes media around between participants while balancing its limitations with the media inputs it receives. With this approach, participants send their media to the server and receive others' media in separate streams, one each.  
For more details of how mediasoup works and how we interacted with it within the browser, check out their documentation:  
- [Server Side Library](https://mediasoup.org/documentation/v3/mediasoup/api/)
- [Client Side Library](https://mediasoup.org/docum

In [ ]:
pretty_print_response("How are messages traded between client, backend and the media server?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: How are messages traded between client, backend and the media server?


Answer:  Based on the provided sources, messages are traded between client, backend, and the media server through the use of rabbitMQ.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: Welcome to CrowdWire
Header: Overview
Source: docs/media_server.md
Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs/media_server.md
Content:
[Mediasoup](https://mediasoup.org/) has an SFU (Selective Forwarding Unit) topology, where the server routes media around between participants while balancing its limitations with the media inputs it receives. With this approach, participants send their media to the server and receive others' media in separate streams, one each.  
For more details of how mediasoup works and how we interacted with it within the browser, check out their documentation:  
- [Server Side Library](ht

## [SQL Tables GH Repo](https://github.com/JannikArndt/sql-auto-doc/tree/master/Examples)

In [ ]:
pretty_print_response("What is the column joined?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What is the column joined?


Answer:  The column joined represents the date when the employee joined our company.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: Welcome to CrowdWire
Header: DB Modelation
Source: docs/REST_API.md
Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs/REST_API.md
Content:
To model our database we used the following diagram:
----------------------------------------------------------------------------------------------------
Document 2 
Title: data.users
Header: data.users
Source: tables.md
Link: https://github.com/JannikArndt/sql-auto-doc/tree/master/Examples//tables.md
Content:
Employees who have software licenses.  
| ID | Name | Type(Length) | Nullable | Default | Example | Comment |
| ---- | ------ | -------------- | ---------- | --------- | ---------------------------- | ----------------------------------------- |
| 1 | id | bigint(8) 

In [ ]:
pretty_print_response("What is the table users?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What is the table users?


Answer:  The table "users" has the following columns:

* ID (bigint(8)) - Primary key and technical column
* Name (nvarchar(800)) - Non-technical column
* Email (nvarchar(800)) - Non-technical column
* Age (int(4)) - Technical column with a default value of 0
* Joined (datetime2(8)) - Technical column with a default value of the current date and time

Note: The table structure and data types may vary based on the specific implementation and requirements of the application.


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: Welcome to CrowdWire
Header: DB Modelation
Source: docs/REST_API.md
Link: https://github.com/Crowd-Wire/ProjectDocumentation/tree/master//docs/REST_API.md
Content:
To model our database we used the following diagram:
----------------------------------------------------------------------------------------------------
Document 2 
Title: How-to open ac

In [ ]:
pretty_print_response("What is the column for the date the software was installed?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Question: What is the column for the date the software was installed?


Answer:  The column for the date the software was installed is "installed".


----------------------------------------------------------------------------------------------------
Documents:

Document 1 
Title: data.software
Header: data.software
Source: data/software.md
Link: https://github.com/JannikArndt/sql-auto-doc/tree/master/Examples//data/software.md
Content:
Software bought for or by the company  
| ID | Name | Type(Length) | Nullable | Default | Example | Comment |
| ---- | --------- | -------------- | ---------- | --------- | ---------------- | ------------------------------------------ |
| 1 | id | int(4) | false | 0 | 5 | Primary Key, technical |
| 2 | name | nvarchar(800) | true | 0 | “Office 2016” | Name of the software |
| 3 | creator | nvarchar(800) | true | 0 | “Microsoft” | Creator behind the software |
| 4 | website | nvarchar(800) | true | 0 | “www.office.com” | Website of the software product o